# 1. ETL

In [1]:
import requests
import geopandas as gpd
from pathlib import Path
import os
import pandas as pd
from shapely.geometry import Point

In [2]:
ROOT = Path(os.path.abspath('')).resolve().parents[0]
DATA = os.path.join(ROOT, "data")
EXTERNAL_DATA = os.path.join(DATA, "external") 
INTERIM_DATA = os.path.join(DATA, "interim")
RAW_DATA = os.path.join(DATA, "raw")


## Load initial dataset

* Full description of dataset is located in [/docs/datensatzbeschreibung.pdf](../docs/datensatzbeschreibung.pdf)
* [Source](https://daten.berlin.de/datensaetze/fahrraddiebstahl-in-berlin)

In [3]:
df = pd.read_csv(
    os.path.join(RAW_DATA, './Bicycle Theft Data.csv'),
    encoding='cp1252'
)

In [4]:
df.columns = df.columns.str.lower()
df.columns

Index(['angelegt_am', 'tatzeit_anfang_datum', 'tatzeit_anfang_stunde',
       'tatzeit_ende_datum', 'tatzeit_ende_stunde', 'lor', 'schadenshoehe',
       'versuch', 'art_des_fahrrads', 'delikt', 'erfassungsgrund'],
      dtype='object')

### Data description

From the source file, this is what explanation for columns exist (transalted to english via ChatGPT):

* `ANGELEGT_AM` → Case creation datetime (UTC+01/CEST as stored)
* `TATZEIT_ANFANG_DATUM` → Offence start date
* `TATZEIT_ANFANG_STUNDE` → Offence start hour
* `TATZEIT_ENDE_DATUM` → Offence end date
* `TATZEIT_ENDE_STUNDE` → Offence end hour
* `LOR` → LOR planning area ID (8-digit PLR code)
* `SCHADENSHOEHE` (SCHADENSHÖHE) → Loss amount (EUR)
* `VERSUCH` → Attempt (yes/no)
* `ART DES FAHRRADS` → Bicycle type
* `DELIKT` → Offence group
* `ERFASSUNGSGRUND` → Recording basis (offence classification)

Rename columns accordingly:

In [5]:
df.columns = [
    'created_at',
    'start_date',
    'start_hour',
    'end_date',
    'end_hour',
    'lor',
    'price',
    'attempt',
    'bicycle_type',
    'group',
    'type'
]

In [6]:
df.dtypes

created_at      object
start_date      object
start_hour       int64
end_date        object
end_hour         int64
lor              int64
price            int64
attempt         object
bicycle_type    object
group           object
type            object
dtype: object

In [7]:
for column in ['created_at', 'start_date', 'end_date']:
    df[column] = pd.to_datetime(df[column], errors='coerce')

/var/folders/v0/8r5h4ym13fb_mzmqc1hfkt580000gn/T/ipykernel_73187/2565069612.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')
/var/folders/v0/8r5h4ym13fb_mzmqc1hfkt580000gn/T/ipykernel_73187/2565069612.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')


In [8]:
df.dtypes

created_at      datetime64[ns]
start_date      datetime64[ns]
start_hour               int64
end_date        datetime64[ns]
end_hour                 int64
lor                      int64
price                    int64
attempt                 object
bicycle_type            object
group                   object
type                    object
dtype: object

In [9]:
df['bicycle_type'].unique()

array(['Damenfahrrad', 'Herrenfahrrad', 'Mountainbike', 'Fahrrad',
       'diverse Fahrräder', 'Kinderfahrrad', 'Lastenfahrrad', 'Rennrad'],
      dtype=object)

In [10]:
df['attempt'].unique()

array(['Nein', 'Ja', 'Unbekannt'], dtype=object)

In [11]:
df['type'].unique()

array(['Sonstiger schwerer Diebstahl von Fahrrädern',
       'Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern',
       'Einfacher Diebstahl von Fahrrädern',
       'Einfacher Diebstahl aus Keller/Boden von Fahrrädern'],
      dtype=object)

In [12]:
df['group'].unique()

array(['Fahrraddiebstahl', 'Keller- und Bodeneinbruch'], dtype=object)

Map column values to english:

In [13]:
bicycle_type_mapping = {
    "Damenfahrrad": "step_through",
    "Herrenfahrrad": "diamond_frame",
    "Mountainbike": "mtb",
    "Fahrrad": "generic",
    "diverse Fahrräder": "multiple",
    "Kinderfahrrad": "kids",
    "Lastenfahrrad": "cargo",
    "Rennrad": "road"
}

group_mapping = {
    "Fahrraddiebstahl": "bicycle_theft",
    "Keller- und Bodeneinbruch": "cellar_attic_burglary"
}

type_mapping = {
    "Sonstiger schwerer Diebstahl von Fahrrädern": "other_aggravated_bicycle_theft",
    "Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern": "other_aggravated_bicycle_theft_cellar_attic",
    "Einfacher Diebstahl von Fahrrädern": "simple_bicycle_theft",
    "Einfacher Diebstahl aus Keller/Boden von Fahrrädern": "simple_bicycle_theft_cellar_attic",
}

attempt_mapping = {
    'Unbekannt': 0,
    'Nein': 1,
    'Ja': 2
}

df['bicycle_type'] = df['bicycle_type'].map(bicycle_type_mapping)
df['group'] = df['group'].map(group_mapping)
df['type'] = df['type'].map(type_mapping)
df['attempt'] = df['attempt'].map(attempt_mapping).astype(int)
df["lor"] = (
    pd.to_numeric(df["lor"], errors="coerce")
      .astype("Int64")
      .astype(str)
      .str.replace(r"\.0$", "", regex=True)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

In [14]:
df

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,type
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,other_aggravated_bicycle_theft
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,other_aggravated_bicycle_theft_cellar_attic
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,other_aggravated_bicycle_theft
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,other_aggravated_bicycle_theft
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,other_aggravated_bicycle_theft
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,other_aggravated_bicycle_theft


In [15]:
df.dtypes

created_at      datetime64[ns]
start_date      datetime64[ns]
start_hour               int64
end_date        datetime64[ns]
end_hour                 int64
lor                     object
price                    int64
attempt                  int64
bicycle_type            object
group                   object
type                    object
dtype: object

### Save dataframe

In [16]:
df.to_parquet(
    os.path.join(INTERIM_DATA, './bicycle_theft_utf8.parquet.gzip'),
    index=False, compression='gzip'
)

## Load LORs map dataset

* [Source](https://daten.odis-berlin.de/de/dataset/lor_planungsgraeume_2021/?utm_source=chatgpt.com)

In [17]:
url = "https://tsb-opendata.s3.eu-central-1.amazonaws.com/lor_planungsgraeume_2021/lor_planungsraeume_2021.geojson"
gdf = gpd.read_file(url)

Transform LOR value - add leading zero for compatibility:

In [18]:
gdf["lor"] = (
    gdf["PLR_ID"]
      .astype(str)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

In [19]:
missing = (df[["lor"]].drop_duplicates()
           .merge(gdf[["lor"]], on="lor", how="left", indicator=True)
           .query("_merge == 'left_only'")["lor"])
print("Was not found in geolayer:", len(missing))
print(missing.head(20).tolist())

Was not found in geolayer: 0
[]


Map code to name of district:

In [20]:
bez_map = {
    "01": "Mitte",
    "02": "Friedrichshain-Kreuzberg",
    "03": "Pankow",
    "04": "Charlottenburg-Wilmersdorf",
    "05": "Spandau",
    "06": "Steglitz-Zehlendorf",
    "07": "Tempelhof-Schöneberg",
    "08": "Neukölln",
    "09": "Treptow-Köpenick",
    "10": "Marzahn-Hellersdorf",
    "11": "Lichtenberg",
    "12": "Reinickendorf",
}
gdf["bez_name"] = gdf["BEZ"].astype(str).str.zfill(2).map(bez_map)

### Save dataframe

In [21]:
gdf.to_parquet(
    os.path.join(EXTERNAL_DATA, 'geo_data.geoparquet.gzip'),
    compression="gzip"
)

### Join map dataframe with initial dataframe

In [22]:
cols_geom = ["lor", "PLR_NAME", "BEZ", 'bez_name', "geometry"]
df_geo = df.merge(gdf[cols_geom], on="lor", how="left")
df_geo.columns = df_geo.columns.str.lower()
df_geo

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,type,plr_name,bez,bez_name,geometry
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,other_aggravated_bicycle_theft,Franziusweg,07,Tempelhof-Schöneberg,"MULTIPOLYGON (((389917.274 5805460.874, 389900..."
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft,Elberfelder Straße,01,Mitte,"MULTIPOLYGON (((387304.093 5820870.943, 387311..."
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,other_aggravated_bicycle_theft_cellar_attic,Humboldthain Nordwest,01,Mitte,"MULTIPOLYGON (((389194.203 5821902.514, 389190..."
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,other_aggravated_bicycle_theft,Rodenbergstraße,03,Pankow,"MULTIPOLYGON (((392843.655 5823122.007, 392841..."
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft,Babelsberger Straße,04,Charlottenburg-Wilmersdorf,"MULTIPOLYGON (((387086.809 5816385.329, 387089..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,other_aggravated_bicycle_theft,Hochmeisterplatz,04,Charlottenburg-Wilmersdorf,"MULTIPOLYGON (((384114.38 5817711.878, 384113...."
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft,Flinsberger Platz,04,Charlottenburg-Wilmersdorf,"MULTIPOLYGON (((383969.491 5817363.025, 383976..."
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,other_aggravated_bicycle_theft,Schweizer Viertel,06,Steglitz-Zehlendorf,"MULTIPOLYGON (((383049.818 5808634.589, 383035..."
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,other_aggravated_bicycle_theft,Oberfeldstraße,10,Marzahn-Hellersdorf,"MULTIPOLYGON (((401663.079 5821243.214, 401663..."


Map code to name of district:

Number of events per district:

In [23]:
df_geo.groupby('bez_name')['created_at'].count().sort_values(ascending=False)

bez_name
Mitte                         2466
Friedrichshain-Kreuzberg      1786
Pankow                        1628
Charlottenburg-Wilmersdorf    1422
Tempelhof-Schöneberg          1158
Neukölln                      1101
Treptow-Köpenick              1079
Steglitz-Zehlendorf            893
Lichtenberg                    837
Reinickendorf                  529
Marzahn-Hellersdorf            365
Spandau                        354
Name: created_at, dtype: int64

## Load weather data

* For weather-related features, let us use [free weather api](https://open-meteo.com/):

In [24]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.52,
    "longitude": 13.405,
    "start_date": "2024-01-01",
    "end_date": "2025-11-04",
    "daily": ",".join([
        "temperature_2m_mean","temperature_2m_min","temperature_2m_max",
        "precipitation_sum","wind_speed_10m_max","sunshine_duration"
    ]),
    "timezone": "Europe/Berlin",
}
j = requests.get(url, params=params, timeout=60).json()
open_meteo_df = pd.DataFrame(j["daily"])
if "sunshine_duration" in open_meteo_df:
    open_meteo_df["sunshine_h"] = open_meteo_df["sunshine_duration"] / 3600.0
open_meteo_df.head()

,time,temperature_2m_mean,temperature_2m_min,temperature_2m_max,precipitation_sum,wind_speed_10m_max,sunshine_duration,sunshine_h
0,2024-01-01,5.3,3.5,7.4,1.9,19.7,17859.23,4.960897
1,2024-01-02,4.6,2.5,7.3,8.5,20.2,0.00,0.000000
2,2024-01-03,8.8,7.3,10.6,10.8,27.8,3789.20,1.052556
3,2024-01-04,3.8,-2.2,7.3,2.8,33.1,0.00,0.000000
4,2024-01-05,0.4,-0.6,0.9,5.4,21.3,0.00,0.000000


In [25]:
open_meteo_df['time'] = pd.to_datetime(open_meteo_df['time'])
open_meteo_df.rename(columns={'time': 'created_at'}, inplace=True)

In [26]:
open_meteo_df.dtypes

created_at             datetime64[ns]
temperature_2m_mean           float64
temperature_2m_min            float64
temperature_2m_max            float64
precipitation_sum             float64
wind_speed_10m_max            float64
sunshine_duration             float64
sunshine_h                    float64
dtype: object

### Save dataframe

In [27]:
open_meteo_df.to_parquet(
    os.path.join(EXTERNAL_DATA, "open_meteo.parquet.gzip"),
    compression='gzip'
)

### Join weather dataframe with initial dataframe

In [28]:
df_geo = df_geo.join(open_meteo_df.set_index('created_at'), on='created_at')

## Load population data

* Full description of dataset is located in [/docs/EWR Datenpool Nov 2023.pdf](../docs/EWR%20Datenpool%20Nov%202023.pdf)
* [Source](https://daten.berlin.de/datensaetze/einwohnerinnen-und-einwohner-in-berlin-in-lor-planungsraumen-am-31-12-2024?utm_source=chatgpt.com)

In [29]:
population_df = pd.read_csv(
    os.path.join(RAW_DATA, 'population.csv'), sep=';'
)
population_df.head(5)

,ZEIT,RAUMID,BEZ,PGR,BZR,PLR,BEZPGR,E_E,E_EM,E_EW,...,E_E95_110,E_EU1,E_E1U6,E_E6U15,E_E15U18,E_E18U25,E_E25U55,E_E55U65,E_E65U80,E_E80U110
0,202412,1100101,1,10,1,1,110,3580.0,1869.0,1711.0,...,3.0,37.0,128.0,203.0,49.0,296.0,1774.0,376.0,455.0,262.0
1,202412,1100102,1,10,1,2,110,2034.0,1113.0,921.0,...,0.0,18.0,73.0,105.0,27.0,144.0,1101.0,304.0,198.0,64.0
2,202412,1100103,1,10,1,3,110,5790.0,3073.0,2717.0,...,13.0,52.0,235.0,374.0,123.0,422.0,2807.0,679.0,780.0,318.0
3,202412,1100104,1,10,1,4,110,4889.0,2587.0,2302.0,...,9.0,43.0,212.0,355.0,129.0,464.0,2680.0,557.0,371.0,78.0
4,202412,1100205,1,10,2,5,110,2917.0,1545.0,1372.0,...,3.0,15.0,97.0,161.0,49.0,238.0,1556.0,310.0,340.0,151.0


In [30]:
population_df = population_df.rename(columns={'RAUMID': 'lor'})
population_df['lor'] = (
    population_df['lor'].astype(str)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

Map keys from documentation to required values:

In [31]:
rename_keys = {
    "ZEIT": "population_snapshot_date",
    "BEZ": "bez_code",
    "PLR": "plr_code",
}

rename_E = {
    "E_E": "population_total",
    "E_EM": "population_male",
    "E_EW": "population_female",
    "E_E00_01": 'age_0_1',
    "E_E14_15": "age_14_15",
    "E_E15_18": "age_15_18",
    "E_E18_21": "age_18_21",
    "E_E25_27": "age_25_27",
    "E_E55_60": "age_55_60",
    'E_E60_63': 'age_60_64',
    "E_E80_85": "age_80_85",
}
columns_to_rename = {**rename_keys, **rename_E}
columns = ['lor'] + list(rename_keys.values()) + list(rename_E.values()) 
columns_to_rename
population_df = population_df.rename(columns=columns_to_rename)[columns]
population_df = population_df.fillna(0.0)
population_df

,lor,population_snapshot_date,bez_code,plr_code,population_total,population_male,population_female,age_0_1,age_14_15,age_15_18,age_18_21,age_25_27,age_55_60,age_60_64,age_80_85
0,01100101,202412,1,1,3580.0,1869.0,1711.0,37.0,24.0,49.0,67.0,170.0,173.0,130.0,151.0
1,01100102,202412,1,2,2034.0,1113.0,921.0,18.0,9.0,27.0,50.0,60.0,175.0,72.0,43.0
2,01100103,202412,1,3,5790.0,3073.0,2717.0,52.0,31.0,123.0,133.0,210.0,333.0,184.0,150.0
3,01100104,202412,1,4,4889.0,2587.0,2302.0,43.0,51.0,129.0,134.0,222.0,262.0,191.0,42.0
4,01100205,202412,1,5,2917.0,1545.0,1372.0,15.0,18.0,49.0,66.0,123.0,143.0,100.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,12601032,202412,12,32,6250.0,3121.0,3129.0,63.0,96.0,262.0,284.0,158.0,362.0,204.0,209.0
538,12601133,202412,12,33,11448.0,5550.0,5898.0,111.0,141.0,494.0,434.0,275.0,659.0,392.0,380.0
539,12601134,202412,12,34,16006.0,7699.0,8307.0,165.0,202.0,577.0,600.0,427.0,945.0,549.0,681.0
540,12601235,202412,12,35,10610.0,5215.0,5395.0,106.0,172.0,477.0,470.0,284.0,659.0,326.0,328.0


In [32]:
population_df.dtypes

lor                          object
population_snapshot_date      int64
bez_code                      int64
plr_code                      int64
population_total            float64
population_male             float64
population_female           float64
age_0_1                     float64
age_14_15                   float64
age_15_18                   float64
age_18_21                   float64
age_25_27                   float64
age_55_60                   float64
age_60_64                   float64
age_80_85                   float64
dtype: object

### Save population dataset

In [33]:
population_df.to_parquet(
    os.path.join(INTERIM_DATA, 'population.parquet.gzip'),
    compression='gzip'
)

### Join population dataset with initial

In [34]:
df_geo = df_geo.join(population_df.set_index('lor'), on='lor')
df_geo

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,...,population_male,population_female,age_0_1,age_14_15,age_15_18,age_18_21,age_25_27,age_55_60,age_60_64,age_80_85
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,...,3419.0,3558.0,34.0,73.0,177.0,218.0,119.0,616.0,368.0,372.0
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,...,5683.0,5838.0,77.0,76.0,234.0,294.0,385.0,772.0,554.0,305.0
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,...,7874.0,7073.0,113.0,146.0,439.0,531.0,645.0,761.0,448.0,226.0
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,...,3446.0,3544.0,60.0,55.0,127.0,114.0,213.0,462.0,207.0,119.0
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,...,3849.0,3714.0,62.0,56.0,231.0,185.0,245.0,510.0,288.0,217.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,...,4024.0,4257.0,57.0,53.0,168.0,177.0,244.0,598.0,315.0,359.0
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,...,3875.0,4447.0,63.0,59.0,173.0,192.0,221.0,593.0,389.0,430.0
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,...,6706.0,6989.0,74.0,173.0,484.0,468.0,249.0,1222.0,678.0,645.0
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,...,3892.0,4034.0,37.0,64.0,185.0,177.0,211.0,614.0,456.0,426.0


## Load trafic dencity dataset

* For loading of traffic density, let us use [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) from [OpenStreetMap](https://wiki.openstreetmap.org/wiki/Main_Page)

In [35]:
query = """
[out:json][timeout:60];
area["name"="Berlin"]["boundary"="administrative"]->.a;
(
  nwr(area.a)["amenity"="bicycle_parking"];
  nwr(area.a)["shop"="bicycle"];
  nwr(area.a)["railway"="station"];
);
out center tags;
"""
r = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
r.raise_for_status()
elements = r.json()["elements"]

# Transform data to points (node -> lon/lat; way/rel -> center.lon/lat)
rows = []
for el in elements:
    tags = el.get("tags", {})
    if el["type"] == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        c = el.get("center")
        if not c:
            continue
        lon, lat = c["lon"], c["lat"]
    rows.append({"id": f"{el['type']}/{el['id']}", "lon": lon, "lat": lat, **tags})

df_pois = pd.DataFrame(rows)

# Classification of the given data
def classify(rec):
    if rec.get("amenity") == "bicycle_parking": return "bike_parking"
    if rec.get("shop") == "bicycle":            return "bike_shop"
    if rec.get("railway") == "station":         return "rail_station"
    return "other"

df_pois["kind"] = df_pois.apply(classify, axis=1)

pois = gpd.GeoDataFrame(
    df_pois,
    geometry=[Point(xy) for xy in zip(df_pois["lon"], df_pois["lat"])],
    crs=4326
).to_crs(25833)

pois = pois.drop_duplicates(subset=["id"]).reset_index(drop=True)

In [36]:
if pois.crs is None:
    pois = pois.set_crs(4326)
pois = pois.to_crs(25833)

# Get one polygon per LOR and build dataframe based on this
lor_polys = (df_geo[["lor", "geometry"]]
             .dropna(subset=["geometry"])
             .drop_duplicates(subset=["lor"])
             .copy())
gdf_lor = gpd.GeoDataFrame(lor_polys, geometry="geometry")

if gdf_lor.crs is None:
    minx, miny, maxx, maxy = gdf_lor.total_bounds
    if max(abs(minx), abs(maxx)) <= 180 and max(abs(miny), abs(maxy)) <= 90:
        gdf_lor = gdf_lor.set_crs(4326)
    else:
        gdf_lor = gdf_lor.set_crs(25833)

gdf_lor_25833 = gdf_lor.to_crs(25833)

joined = gpd.sjoin(
    pois,
    gdf_lor_25833[["lor", "geometry"]],
    how="inner",
    predicate="within"
)

area_km2 = (gdf_lor_25833.set_index("lor").area / 1e6).rename("area_km2")
poi_cnt = joined.groupby("lor").size().rename("poi_cnt")

poi_density = (poi_cnt.to_frame()
               .join(area_km2, how="right")
               .fillna({"poi_cnt": 0}))
poi_density["poi_density_km2"] = poi_density["poi_cnt"] / poi_density["area_km2"]
poi_density

,poi_cnt,area_km2,poi_density_km2
lor,,,
07601546,6.0,1.773447,3.383241
01200522,128.0,0.639144,200.267843
01300836,100.0,1.440714,69.410022
03601243,71.0,0.329525,215.461750
04501153,65.0,0.585827,110.954225
...,...,...,...
09301227,1.0,2.355437,0.424550
04400831,0.0,0.544449,0.000000
04400725,0.0,0.543418,0.000000


### Save traffic density dataset

In [37]:
poi_density.to_parquet(
    os.path.join(EXTERNAL_DATA, 'traffic_density.parquet.gzip'),
    compression='gzip'
)

### Join traffic density dataset with initial

In [38]:
df_geo = df_geo.join(poi_density, on='lor')

## Save combined data

In [39]:
# Cast to GeoDataFrame so that it is possible to save it in .geoparquet format
gdf = gpd.GeoDataFrame(df_geo, geometry="geometry", crs=gdf.crs or 25833)
gdf.to_parquet(
    os.path.join(INTERIM_DATA, 'df_geo_etl.geoparquet.gzip'),
    compression='gzip'
)